In [1]:
import torch

In [1]:
# from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_community.document_loaders import OnlinePDFLoader
from langchain_community.document_loaders import PyPDFLoader

In [2]:
local_path = "input.pdf"

# Local PDF file uploads
if local_path:
  loader = PyPDFLoader(file_path=local_path)
  data = loader.load()
else:
  print("Upload a PDF file")

In [3]:
# Preview first page
data[0].page_content

'1.1 WHAT IS PHYSICS ?\nThe nature around us is colourful and diverse. It\ncontains phenomena of large varieties. The winds, the\nsands, the waters, the planets, the rainbow, heating of\nobjects on rubbing, the function of a human body, the\nenergy coming from the sun and the nucleus ……  there\nare a large number of objects and events taking place\naround us.\nPhysics is the study of nature and its laws. We\nexpect that all these different events in nature take\nplace according to some basic laws and revealing these\nlaws of nature from the observed events  is physics. For\nexample, the orbiting of the moon around the earth,\nfalling of an apple from a tree and tides in a sea on a\nfull moon night can all be explained if we know the\nNewton’s law of gravitation and Newton’s laws of\nmotion. Physics is concerned with the basic rules\nwhich are applicable to all domains of life.\nUnderstanding of physics, therefore, leads to\napplications in many fields including bio and medical\nscience

In [4]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [5]:
# Split and chunk 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

In [7]:
vector_db = Chroma.from_documents(
    documents=chunks, 
    embedding=OllamaEmbeddings(model="nomic-embed-text",show_progress=True),
    collection_name="local-rag"
)

OllamaEmbeddings: 100%|██████████| 455/455 [00:07<00:00, 59.55it/s]


In [8]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

In [9]:
local_model = "llama3.2"
llm = ChatOllama(model=local_model)

/tmp/ipykernel_25316/811129129.py:2: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model=local_model)


In [10]:
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate five
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

In [11]:
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

# RAG prompt
template = """Answer the question based ONLY on the following context:
{context}
Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [12]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [18]:
chain.invoke(input(""))

OllamaEmbeddings: 100%|██████████| 1/1 [00:00<00:00, 243.39it/s]


'To solve this problem, we need to use the concept of centripetal force and Newton\'s second law.\n\nLet\'s break down the problem step by step:\n\n1. Convert the speed from km/hr to m/s:\n900 km/hr = 900 × (1000/3600) m/s = 250 m/s\n\n2. The fighter plane is moving in a vertical circle, so it experiences a centripetal force directed towards the center of the circle. We can find this force using the equation:\n\nFc = (m × v^2) / r\n\nwhere Fc is the centripetal force, m is the mass of the plane, v is its velocity, and r is the radius of the circle.\n\n3. Substitute the given values:\nFc = (16000 kg × (250 m/s)^2) / 2000 m\n= 16000 × 62500 / 2000\n= 10,000,000 N\n\n4. However, this is not the force exerted by the air on the plane at the lowest point. To find this force, we need to consider the component of the centripetal force that acts in the direction of gravity (i.e., downwards). We can do this by finding the component of Fc along the gravitational field.\n\nLet\'s call the angle be

In [ ]:
chain.invoke(input(""))

In [ ]:
vector_db.delete_collection()